In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.stats.weightstats as ws

import os 

import sys; sys.path.append('/home/mitch/util/python')
sys.path.append('/home/mitch/school/mfl2/src/')

import panel_distribution_utils as pdu
from tabulate import tabulate


import utils

save = False

In [11]:
raw     = '/home/mitch/Dropbox/data/mexico_fls/raw/2005/'
interim = '/home/mitch/Dropbox/data/mexico_fls/interim/2005/'

figs    = '/home/mitch/school/mfl2/fig/'
dicts   = '/home/mitch/school/mfl2/notebooks/dicts/'

In [21]:
os.chdir(interim)
data = pd.read_stata('merged_2005.dta')

In [22]:
data = data.query('annual_labor_income_hh > 0')

In [23]:
q01 = ws.DescrStatsW(data['liquid_formal_hh'], weights=data['weight_hh']).quantile(0.01).values[0]
q99 = ws.DescrStatsW(data['liquid_formal_hh'], weights=data['weight_hh']).quantile(0.99).values[0]
data = data[data['liquid_formal_hh'] > q01]
data = data[data['liquid_formal_hh'] < q99]

In [24]:
data['liquid_formal_hh']
stats = ws.DescrStatsW(data['annual_labor_income_hh'], data['weight_hh'])
bound = data['annual_labor_income_hh'] <= stats.quantile(0.90).values[0]
total_liquid_wealth = np.sum(data['liquid_formal_hh'] * data['weight_hh'])
liquid_wealth_bottom_50 = np.sum(data['liquid_formal_hh'][bound] * data['weight_hh'][bound])
share = liquid_wealth_bottom_50 / total_liquid_wealth
print(share)


0.6496732965490566


In [25]:
import importlib
importlib.reload(pdu)
share_top_05 = 1 - pdu.share(0.95, data['liquid_formal_hh'], data['annual_labor_income_hh'], data['weight_hh'] )
share_top_10 = 1 - pdu.share(0.90, data['liquid_formal_hh'], data['annual_labor_income_hh'], data['weight_hh'] )
share_bottom_50 = pdu.share(0.50, data['liquid_formal_hh'], data['annual_labor_income_hh'], data['weight_hh'] )
share_bottom_25 = pdu.share(0.25, data['liquid_formal_hh'], data['annual_labor_income_hh'], data['weight_hh'] )
gini = pdu.gini(data['weight_hh'].astype('float64'), data['liquid_formal_hh'])

names = ['share_top_05', 'share_top_10', 'share_bottom_50', 'share_bottom_25', 'gini']
results = [share_top_05, share_top_10, share_bottom_50, share_bottom_25, gini]
print(tabulate({'Stat':names, 'Result':results}))

---------------  ---------
share_top_05     0.295088
share_top_10     0.350327
share_bottom_50  0.147506
share_bottom_25  0.0227671
---------------  ---------


In [35]:
import importlib
importlib.reload(pdu)
share_top_05 = 1 - pdu.share(0.95, data['illiquid_assets'], data['annual_labor_income_hh'], data['weight_hh'] )
share_top_10 = 1 - pdu.share(0.90, data['illiquid_assets'], data['annual_labor_income_hh'], data['weight_hh'] )
share_bottom_50 = pdu.share(0.50, data['illiquid_assets'], data['annual_labor_income_hh'], data['weight_hh'] )
share_bottom_25 = pdu.share(0.25, data['illiquid_assets'], data['annual_labor_income_hh'], data['weight_hh'] )

names = ['share_top_05', 'share_top_10', 'share_bottom_50', 'share_bottom_25']
results = [share_top_05, share_top_10, share_bottom_50, share_bottom_25]
print(tabulate({'Stat':names, 'Result':results}))

---------------  ---------
share_top_05     0.0518014
share_top_10     0.12008
share_bottom_50  0.42016
share_bottom_25  0.208112
---------------  ---------


In [19]:
data['illiquid_assets']

TypeError: list indices must be integers or slices, not str

In [12]:
np.sum(((data['liquid_formal_hh'] < 0) + 0.0) * data['weight_hh']) / np.sum(data['weight_hh'])

0.07046154672318422

In [36]:
ill = np.sum(data['illiquid_assets'] * data['weight_hh'])
income = np.sum(data['annual_labor_income_hh'] * data['weight_hh'])
ill / income

1.1022053106055119

# todo tomorrow: normalize by number of persons in household?